# Stack Overflow Survey 2022
### Using the ordinary least squares (OLS) regression, it was analyzed which factors have the greatest impact on the salary

Stack Overflow is a question and answer website for developers. Stack Overflow conducts a survey every year in order to help them to improve the Stack Overflow community and the platform.
The goal of this analysis is to find out what **affects the developer's salary** the most with the help of the **least squares method (OLS)**. The survey has 79 questions, and therefore this data has 79 columns, for each question 1 column. There are many missing values in some columns and those columns will be dropped. Below is an explanation of the columns that will be used for the OLS method.

*Definitions*
- **Gender** - user's gender
- **Country** - country where the user lives
- **OrgSize** - number of people that are employed by the company or organization the user currently work for
- **Age**  - user's age
- **EdLevel** - the highest level of formal education that user have completed
- **MainBranch** - is coding a main branch or a hobby
- **Employment** - employment status; full-time, part-time...
- **RemoteWork** - does the user work from home or office
- **CodingActivities** - does user writes code outside of work
- **LearnCode** - how did user learn to code
- **YearsCode** - how many years does user code (including education)
- **YearsCodePro** - how many years does user code (not including education)
- **DevType** - type of developer, profession
- **PurchaseInfluence** - level of influence that user have over new technology purchases at organization
- **LanguageHaveWorkedWith** - programming, scripting, and markup languages that user have done extensive development work in over the past year
- **NEWCollabToolsHaveWorkedWith** - development environments that user used over the past year
- **OpSysPersonal_use** - primary operating system in which user work
- **VersionControlSystem** - primary version control systems that user use
- **NEWSOSites** - Stack Overflow sites that user have visited
- **SOVisitFreq** - how frequently user visits Stack Overflow
- **SOAccount** - does user have a Stack Overflow account
- **CompYearly** - the current total compensation (salary, bonuses, and perks, before taxes and deductions)
- **Currency** - currency used by the user

# Imports and Reading Data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 80)
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
import statsmodels.api as sm

In [2]:
survey = pd.read_csv('survey_results_public_2022.csv')

# Cleaning Data

The first step in cleaning the dataframe is to **filter** the dataframe to have the relevant data. First, we will remove all rows that have a value *'None of these'* in the *MainBranch* column, because all other columns have missing values, since these users **don't code**. We will also drop rows where users entered 0 as salary value. All missing values in column *CompTotal* will be dropped because on that column the OLS analysis will be applied. We will also drop some unnecessary columns that are irrelevant for this analysis like *SurveyEase and SurveyLength* because those questions are about survey. After that we will drop all columns that have **more than 500 missing values**, and the rest of the missing values in dataframe will be dropped by **row**. 

After that we will adjust all salaries so that they are on the **same basis**. The salary amount could be entered on an *annual, monthly and weekly basis*. We will adjust everything to the **annual basis**. Also, not all users used the same currency when entering the salary amount. Based on the *Currency* column, we will assume that the salaries are shown in the given currency. We will convert all salaries into **euros**. Currency rates are uploaded from *json file* which was web scraped with Selenium (**Currency Rate Project**). 

In [3]:
def transformData(df):
    
    dictionary = {'More than 50 years': 50, 'Less than 1 year':0.5}

    def prepareData(df):
        return (df
     .query('MainBranch != "None of these" & CompTotal != 0.0')
     .assign(YearsCode = df.YearsCode.replace(dictionary), #adjusting all to numbers
             YearsCodePro = df.YearsCodePro.replace(dictionary), #adjusting all to numbers
             Currency = df.Currency.str[:3] #getting only currency code
     )
     .dropna(subset='CompTotal', axis=0) #dropping missing values in salary column
     .drop(columns=(['ResponseId', 'SurveyLength', 'SOComm', 'SurveyEase', 
                     'ConvertedCompYearly', 'Blockchain', 'Ethnicity']), axis=1) #dropping irrelevant columns
     .astype({'YearsCode':'float', 'YearsCodePro':'float'})
    )

    survey = prepareData(df)

    survey = survey.loc[:, (survey.isna().sum() < 500)] \
            .dropna(how='any', axis=0) #dropping columns with more than 500 missing values

    currency_dict = json.load(open("currency_dict.json")) #loading file with currency rates

    survey['ExchangeRate'] = survey.Currency.map(currency_dict).astype(float) #converting all currencies to euro
    survey['CompAdjusted'] = survey.ExchangeRate * survey.CompTotal #converting all salaries into euro

    #adjusting all salaries to annual basis
    survey.loc[survey.CompFreq == 'Yearly', 'CompYearly'] = survey.CompAdjusted * 1
    survey.loc[survey.CompFreq == 'Monthly', 'CompYearly'] = survey.CompAdjusted * 12
    survey.loc[survey.CompFreq == 'Weekly', 'CompYearly'] = survey.CompAdjusted * 52

    def cleanData(survey):
        return (survey
     .query('CompYearly < 500000') #dropping extreme outliers, above 500000€
     .drop(columns=['Currency', 'CompTotal', 'CompFreq','ExchangeRate', 'CompAdjusted'], axis=1) #dropping unnecessary columns 
     .dropna(how='any', axis=0) #dropping all missing values
     .reset_index(drop=True))

    sur = cleanData(survey)
    return sur

In [4]:
sur = transformData(survey)
#sur.memory_usage(deep=True).sum()

### DevType - Profession

*DevType* column has **29 unique values** which is a lot for doing OLS analysis. In this section we will **re-code** the column on some values. We will put the values in the **same group**. For example, *Developer, full-stack, Developer, back-end, Developer, front-end will* be in the same group *Developer*. In the end, there will be **12 unique values**.

In [5]:
# re-grouping values
Developer = ['Developer, full-stack', 'Developer, back-end', 'Developer, front-end', 
             'Developer, desktop or enterprise applications', 'Developer, mobile', 
             'Developer, embedded applications or devices', 'Developer, game or graphics', 'Developer, QA or test']
DataScientist = ['Data scientist or machine learning specialist', 'Scientist']
Manager = ['Product manager',  'Project manager', 'Engineering manager']
Engineer = ['Cloud infrastructure engineer', 'Engineer, site reliability', 'Engineer, data']  


developer_df = pd.DataFrame(sur.DevType.str.split(';').explode().replace(dict.fromkeys(Developer, 'Developer')) \
         .replace(dict.fromkeys(DataScientist, 'Data scientist')).replace(dict.fromkeys(Manager, 'Manager')) \
         .replace(dict.fromkeys(Engineer, 'Engineer')).replace('Academic researcher', 'Educator') \
         .replace('Database administrator', 'System administrator')) \
         .reset_index()

In [6]:
#dropping irrelevant values, a sample size is too small
dev_list = ['Blockchain', 'Student', 'Marketing or sales professional']

developer_df = developer_df[~developer_df.DevType.isin(dev_list)]

In [7]:
devtype_clean = developer_df.groupby(['index'])['DevType'] \
                            .apply(set) \
                            .apply(list) \
                            .str.join(';')

sur['DevType'] = devtype_clean

### Employment

*Employment* column has **11 unique values** but because of the multiple-choice option there are in total 11 values. From multiple answers, we will reduce it to one answer. For example, if the answer contains full-time, we will put that the user works full-time. The same logic wil be applied for part-time and contractor. In the end, there will be **3 unique values**.

In [8]:
conditions = [
            sur.Employment.str.contains('full-time'), 
            sur.Employment.str.contains('part-time'), sur.Employment.str.contains('contractor')
]


choices = ['Employed, full-time', 'Employed, part-time', 
           'Independent contractor, freelancer, or self-employed']



sur['Employment'] = np.select(conditions, choices, sur['Employment'])

### Country

*Country* column has **156 unique values** and we will reduce it by replacing it with the continent in which user lives. In the end, there will be **3 unique values**. We will upload webpage with the list of countries and continents to map matching continent for each country the users entered. In the end, there will be **7 unique values**.

In [9]:
#uploading table from web
world_table = pd.read_html('https://statisticstimes.com/geography/countries-by-continents.php')[2]

In [10]:
#locating only continent and country column
world_table = world_table.loc[:, ['Country or Area', 'Continent']]

In [11]:
#replacing values
sur = sur.replace(world_table.set_index('Country or Area')['Continent'])

In [12]:
# replacing the rest of countries with continents that aren't included in the table
Europe = ['Czech Republic', 'The former Yugoslav Republic of Macedonia', 'Kosovo']
South_America = ['Bolivia', 'Venezuela, Bolivarian Republic of...']
Asia = ['Taiwan', 'South Korea', 'Hong Kong (S.A.R.)', 'Palestine', 'Iran, Islamic Republic of...']
Africa = ['South Africa', "Côte d'Ivoire", 'Cape Verde', 'Congo, Republic of the...']

sur.Country = sur.Country.replace(dict.fromkeys(Europe, 'Europe')) \
                         .replace(dict.fromkeys(South_America, 'South America')) \
                         .replace(dict.fromkeys(Asia, 'Asia')) \
                         .replace(dict.fromkeys(Africa, 'Africa'))

### Language Have Worked With

*LanguageHaveWorkedWith* column has **42 unique values** and we will reduce it to the languages that have more that **4000 occurrences** and all other values will be dropped. This question in a survey had **multiple-choice** option so first we will count the number of languages the users entered in. In the end, there will be **13 unique values**.

In [13]:
#creating column with number of languages for each user
sur['LanguageNum'] = sur.LanguageHaveWorkedWith.str.split(';').str.len()

In [14]:
#sur['LanguageHaveWorkedWith'].str.split(';').explode().value_counts()

language_filtered = sur.LanguageHaveWorkedWith.str.split(';').explode().value_counts() < 4000 #filtering values

language_list = language_filtered.index[language_filtered].tolist() #creating a list of values that will be dropped

In [15]:
language_values = sur.LanguageHaveWorkedWith.str.split(';').explode() #splitting column values on semicolon

language_bool = language_values.isin(language_list).reset_index(drop=True) #boolean series

In [16]:
language_df = pd.DataFrame(language_values).reset_index()

language_df['Language_bool'] = language_bool

In [17]:
# getting values that have occurrences more than 4000
language_df = language_df[language_df.Language_bool == False]

In [18]:
sur['LanguageHaveWorkedWith'] = language_df.groupby('index')['LanguageHaveWorkedWith'].apply(list).str.join(';')

#sur['LanguageHaveWorkedWith'].str.split(';').explode().value_counts()

### NEWCollabToolsHaveWorkedWith

*NEWCollabToolsHaveWorkedWith* column has **27 unique values** and we will reduce it to the tools that have more that **3000 occurrences** and all other values will be dropped. This question in a survey had **multiple-choice** option so first we will count the number of languages the users entered in. We will apply same steps as for *DevType* column. In the end, there will be **11 unique values**.

In [19]:
#creating column with number of languages for each user
sur['NEWCollabNum'] = sur.NEWCollabToolsHaveWorkedWith.str.split(';').str.len()

In [20]:
#sur['NEWCollabToolsHaveWorkedWith'].str.split(';').explode().value_counts()

NEWCollab_filtered = sur.NEWCollabToolsHaveWorkedWith.str.split(';').explode().value_counts() < 3000 #filtering values

NEWCollab_list = NEWCollab_filtered.index[NEWCollab_filtered].tolist() #creating a list of values that will be dropped

In [21]:
NEWCollab_values = sur.NEWCollabToolsHaveWorkedWith.str.split(';').explode() #splitting column values on semicolon

NEWCollab_bool = NEWCollab_values.isin(NEWCollab_list).reset_index(drop=True) #boolean series

In [22]:
NEWCollab_df = pd.DataFrame(NEWCollab_values).reset_index()

NEWCollab_df['NEWCollab_bool'] = NEWCollab_bool

In [23]:
NEWCollab_df = NEWCollab_df[NEWCollab_df.NEWCollab_bool == False] # getting values that have occurrences more than 4000

sur['NEWCollabToolsHaveWorkedWith'] = NEWCollab_df.groupby('index')['NEWCollabToolsHaveWorkedWith'].apply(list).str.join(';')

#sur['NEWCollabToolsHaveWorkedWith'].str.split(';').explode().value_counts()

### Gender

*Gender* column has **5 unique values** but because of the multiple-choice option there are in total 15 values. From multiple answers, we will reduce it to one answer. For example, if the answer contains *'Man'*, we will put that user is man. The same logic wil be applied for *woman and non-binary*. Values *Prefer not to say and Or, in your own words:* will be put in the same group *'Prefer not to say'*. In the end, there will be **4 unique values**.

In [24]:
#sur['Gender'].value_counts()

cond = [sur.Gender.str.contains('Man'), sur.Gender.str.contains('Woman'), 
           sur.Gender.str.contains('Non-binary'), (sur.Gender.str.contains('Prefer') | sur.Gender.str.contains('words'))]

val = ['Man', 'Woman', 'Non-binary, genderqueer, or gender non-conforming', 'Prefer not to say']

sur['Gender'] = np.select(cond, val, sur.Gender)

### OpSysPersonal_use

*OpSysPersonal use* column has **6 unique values** but because of the multiple-choice option there are in total 54 values. We will drop values *'BSD' and 'Other (please specify):'* because they have the sample size is too small. In the end, there will be **4 unique values**.

In [25]:
OpSys_df = pd.DataFrame(sur['OpSysPersonal use'].str.split(';').explode())

OpSys_df = OpSys_df.reset_index() \
                .set_index(['OpSysPersonal use']) \
                .drop(['Other (please specify):', 'BSD']) \
                .reset_index()

sur['OpSysPersonal use'] = OpSys_df.groupby('index')['OpSysPersonal use'].apply(list).str.join(';')

### VersionControlSystem

*VersionControlSystem* column has **5 unique values** but because of the multiple-choice option there are in total 16 values. We will drop values *'Mercurial' and 'I don't use one'* because they have the least number of occurrences. In the end, there will be **3 unique values**.

In [26]:
VersionControl_df = pd.DataFrame(sur.VersionControlSystem.str.split(';').explode())

VersionControl_df = VersionControl_df.reset_index() \
                .set_index('VersionControlSystem') \
                .drop(['Mercurial', "I don't use one"]) \
                .reset_index()

sur['VersionControlSystem'] = VersionControl_df.groupby('index')['VersionControlSystem'].apply(list).str.join(';')

# Preparing Data

In this part we will make the final preparations before creating dummy variables and applying the OLS method. Assuming that there are still outliers in the **CompYearly** (salary) column, we will filter out all values that are less than *1000* (€). Then we will fill in the missing values in *DevType* and  *VersionControlSystem* with the value *Other*.
*Trans* column will be removed (the question about whether the user is transsexual or not) because there is a big difference between the value occurrences, which could affect the results of the analysis.

Next, we will drop all values that have a small number of occurrences in columns *EdLevel, Age, Gender, Continent, OrgSize* because the sample size is too small.

In [27]:
sur = (sur
 .query('CompYearly > 1000') #filtering out outliers
 .assign(CompYearly = sur.CompYearly.round(2), 
         DevType = sur.DevType.fillna('Other (please specify):'), #filling in missing values
         VersionControlSystem = sur.VersionControlSystem.fillna('Other (please specify):')) #filling in missing values
 .rename(columns={'Country': 'Continent', 'OpSysPersonal use': 'OpSysPersonal_use'})
 .drop(columns=['Trans'], axis=1)
 .dropna(subset=['LanguageHaveWorkedWith', 'NEWCollabToolsHaveWorkedWith', 'OpSysPersonal_use'])
 .reset_index(drop=True)
 .astype({'MainBranch': 'category', 'RemoteWork': 'category', 'OrgSize': 'category', 
          'PurchaseInfluence': 'category', 'SOVisitFreq': 'category', 'SOAccount': 'category'}))

In [28]:
#dropping small smaple size
sur = sur.set_index('EdLevel') \
                .drop(['Professional degree (JD, MD, etc.)', 'Something else', 'Primary/elementary school']) \
                .reset_index()

In [29]:
#dropping small smaple size
sur = sur.set_index('Age') \
                .drop(['65 years or older', 'Prefer not to say', 'Under 18 years old']) \
                .reset_index()

In [30]:
#dropping small smaple size
sur = sur.set_index('OrgSize') \
                .drop(['I don’t know']) \
                .reset_index()

In [31]:
#dropping small smaple size
sur = sur.set_index('Continent') \
                .drop(['Nomadic']) \
                .reset_index()

In [32]:
#dropping small smaple size
sur = sur.set_index('Gender') \
            .drop(['Prefer not to say', 'Non-binary, genderqueer, or gender non-conforming']) \
            .reset_index()

In [33]:
#changing types in category
sur = sur.astype({'EdLevel': 'category', 'OrgSize': 'category', 'Continent':'category', 'Gender':'category'})

#sur.memory_usage(deep=True).sum()

# Creating Dummy Variables

In this part we will create **dummy variables** for all columns. **Numerical and categorical** columns that didn't have the multiple-choice option will be converted into dummy variables using function **pd.get_dummies**. For the columns that had multiple-choice option, (*DevType, CodingActivities, LearnCode, LanguageHaveWorkedWith, NEWCollabToolsHaveWorkedWith, OpSysPersonal use, VersionControlSystem, NEWSOSites*) the new column for each value separately will be created. Two functions are created *(getValues, fillNans)* which will be applied to each column.

In [34]:
#creating new dataframe with dummy variables

sur_dummies = pd.get_dummies(sur, columns= ['MainBranch', 'Employment', 'RemoteWork', 'EdLevel', 'OrgSize', 'PurchaseInfluence',
                'Continent', 'SOVisitFreq', 'SOAccount', 'Gender', 'Age'], drop_first=True)

### DevType dummies

In [35]:
#splitting values on semicolon
DevType_df = pd.DataFrame(sur.DevType.str.split(';') \
                          .explode()) \
                          .replace(['Other (please specify):'], 'Other')

In [36]:
#setting up all unique values as columns and filling in with missing values
def getValues(df, col):
    
    for val in df[col].value_counts().keys():
        df[val] = np.nan
    return df

DevType_val = getValues(DevType_df, 'DevType')

In [37]:
#filling in missing values with 1 when the column name appears in original column, 0 if there's no value
def fillNans(df, col2):
    
    for col in df:
        if df[col].isna().sum()>0:
            c = str(df[col].name)
            df[col] = df[col].fillna(1).where(df[col2].isin([c]))
            df[col] = df[col].fillna(0)
    return df

In [38]:
DevType_val = fillNans(DevType_val, 'DevType')

In [39]:
#grouping by the index
DevType_val = DevType_val.reset_index()

DevType_val = DevType_val.groupby('index')[['Developer', 'Engineer', 'System administrator',
       'DevOps specialist', 'Manager', 'Data scientist',
       'Educator', 'Designer', 'Data or business analyst',
       'Other', 'Security professional',
       'Senior Executive (C-Suite, VP, etc.)']].sum().add_prefix('DevType_')

### CodingActivities dummies

In [40]:
CodingActivities_df = pd.DataFrame(sur.CodingActivities.str.split(';') \
                                   .explode()) \
                                   .replace(['Other (please specify):'], 'Other')

CodingActivities_val = getValues(CodingActivities_df, 'CodingActivities')

In [41]:
CodingActivities_val = fillNans(CodingActivities_val, 'CodingActivities')

In [42]:
CodingActivities_val = CodingActivities_val.reset_index()

In [43]:
CodingActivities_val = CodingActivities_val.groupby('index')[['Hobby',
       'Contribute to open-source projects', 'Freelance/contract work',
       'Bootstrapping a business', 'I don’t code outside of work',
       'School or academic work', 'Other']].sum().add_prefix('CodingActivities_')

### LearnCode dummies

In [44]:
LearnCode_df = pd.DataFrame(sur.LearnCode.str.split(';') \
                            .explode()) \
                            .replace(['Other (please specify):'], 'Other')

LearnCode_val = getValues(LearnCode_df, 'LearnCode')

In [45]:
LearnCode_val = fillNans(LearnCode_val, 'LearnCode')

In [46]:
LearnCode_val = LearnCode_val.reset_index()

In [47]:
LearnCode_val = LearnCode_val.groupby('index')[['Other online resources (e.g., videos, blogs, forum)',
       'School (i.e., University, College, etc)', 'Books / Physical media',
       'On the job training', 'Online Courses or Certification', 'Colleague',
       'Friend or family member', 'Coding Bootcamp',
       'Hackathons (virtual or in-person)', 'Other']].sum().add_prefix('LearnCode_')

### LanguageHaveWorkedWith dummies

In [48]:
LanguageHaveWorkedWith_df = pd.DataFrame(sur.LanguageHaveWorkedWith.str.split(';').explode())         

LanguageHaveWorkedWith_val = getValues(LanguageHaveWorkedWith_df, 'LanguageHaveWorkedWith')

In [49]:
LanguageHaveWorkedWith_val = fillNans(LanguageHaveWorkedWith_val, 'LanguageHaveWorkedWith')

In [50]:
LanguageHaveWorkedWith_val = LanguageHaveWorkedWith_val.reset_index()

In [51]:
LanguageHaveWorkedWith_val = LanguageHaveWorkedWith_val.groupby('index')[['JavaScript', 'HTML/CSS', 'SQL',
       'Python', 'TypeScript', 'Bash/Shell', 'Java', 'C#', 'PHP', 'C++',
       'PowerShell', 'C', 'Go']].sum().add_prefix('LangHaveWorked_')

### NEWCollabToolsHaveWorkedWith dummies

In [52]:
NEWCollabToolsHaveWorkedWith_df = pd.DataFrame(sur.NEWCollabToolsHaveWorkedWith.str.split(';').explode())


NEWCollabToolsHaveWorkedWith_val = getValues(NEWCollabToolsHaveWorkedWith_df, 'NEWCollabToolsHaveWorkedWith')

In [53]:
NEWCollabToolsHaveWorkedWith_val = fillNans(NEWCollabToolsHaveWorkedWith_val, 'NEWCollabToolsHaveWorkedWith')

In [54]:
NEWCollabToolsHaveWorkedWith_val = NEWCollabToolsHaveWorkedWith_val.reset_index()

In [55]:
NEWCollabToolsHaveWorkedWith_val = NEWCollabToolsHaveWorkedWith_val.groupby('index')[['Visual Studio Code',
       'Visual Studio', 'IntelliJ', 'Notepad++', 'Vim', 'Android Studio',
       'Sublime Text', 'PyCharm', 'Xcode', 'Eclipse', 'IPython/Jupyter']].sum().add_prefix('NEWCTHaveWorked_')

### OpSysPersonal use dummies

In [56]:
OpSysPersonal_df = pd.DataFrame(sur.OpSysPersonal_use.str.split(';') \
                                .explode())

OpSysPersonal_val = getValues(OpSysPersonal_df, 'OpSysPersonal_use')

In [57]:
OpSysPersonal_val = fillNans(OpSysPersonal_val, 'OpSysPersonal_use')

In [58]:
OpSysPersonal_val = OpSysPersonal_val.reset_index()

In [59]:
OpSysPersonal_val = OpSysPersonal_val.groupby('index')[['Windows', 'Linux-based', 'macOS',
       'Windows Subsystem for Linux (WSL)']].sum().add_prefix('OpSys_')

### VersionControlSystem dummies

In [60]:
VersionControl_df = pd.DataFrame(sur.VersionControlSystem.str.split(';') \
                                       .explode()) \
                                       .replace(['Other (please specify):'], 'Other')

VersionControl_val = getValues(VersionControl_df, 'VersionControlSystem')

In [61]:
VersionControl_val = fillNans(VersionControl_val, 'VersionControlSystem')

In [62]:
VersionControl_val = VersionControl_val.reset_index()

In [63]:
VersionControl_val = VersionControl_val.groupby('index')[['Git', 'SVN',
       'Other']].sum().add_prefix('VersionControl_')

### NEWSOSites dummies

In [64]:
NEWSOSites_df = pd.DataFrame(sur.NEWSOSites.str.split(';').explode()).replace(
             ['Stack Overflow for Teams (private knowledge sharing & collaboration platform for companies)'], 
             ['Stack Overflow for Teams'])

NEWSOSites_val = getValues(NEWSOSites_df, 'NEWSOSites')

In [65]:
NEWSOSites_val = fillNans(NEWSOSites_val, 'NEWSOSites')

In [66]:
NEWSOSites_val = NEWSOSites_val.reset_index()

In [67]:
NEWSOSites_val = NEWSOSites_val.groupby('index')[['Stack Overflow', 'Stack Exchange',
       'Collectives on Stack Overflow',
       'Stack Overflow for Teams']].sum().add_prefix('NEWSOSites_')

In [68]:
#joining all dummy columns together in a dataframe sur_dummies
dfs_list = [DevType_val, CodingActivities_val, LearnCode_val, LanguageHaveWorkedWith_val, 
           NEWCollabToolsHaveWorkedWith_val, OpSysPersonal_val, VersionControl_val, NEWSOSites_val]

sur_dummies = sur_dummies.join(dfs_list).drop(columns=['DevType', 'CodingActivities', 'LearnCode', 'LanguageHaveWorkedWith', 
           'NEWCollabToolsHaveWorkedWith', 'OpSysPersonal_use', 'VersionControlSystem', 'NEWSOSites'], axis=1)

# OLS Analysis

Finally, in this part we will apply the **OLS method** to the dataframe in order to find out which factors are **statistically significant** and affect the salary and which are not. First we will **normalize** data by using *preprocessing.MinMaxScaler()* method from **sklearn** library. Next we will define our **x and y variable**. Y variable is the column **CompYearly** (salary), x is the rest of the dataframe after dropping the y. After defining x and y, data is ready for applying OLS analysis. 

In [69]:
# transform features by scaling each feature from 0 to 1
# with the 0 becoming the min score and 1 becoming max score
# everything will vary between those values
# it doesn't change categorical variables because they are already set to 0 and 1

sur_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(sur_dummies), columns=sur_dummies.columns)

In [70]:
# defining x and y variable
y = sur_minmax.CompYearly
x = sur_minmax.drop(columns=['CompYearly'], axis=1).assign(const=1)

In [71]:
# OLS method
results = sm.OLS(y,x).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             CompYearly   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     185.1
Date:                Thu, 20 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:40:35   Log-Likelihood:                 24671.
No. Observations:               30645   AIC:                        -4.913e+04
Df Residuals:                   30540   BIC:                        -4.826e+04
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================================================
                                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------
YearsCode                                                                                      0.0237      0.009      2.679      0.007       0.006       0.041
YearsCodePro                                                                                   0.1363      0.011     12.612      0.000       0.115       0.157
LanguageNum                                                                                    0.1097      0.026      4.217      0.000       0.059       0.161
NEWCollabNum                                                                                  -0.0279      0.019     -1.470      0.142      -0.065       0.009
MainBranch_I am not primarily a developer, but I write code sometimes as part of my work      -0.0127      0.003     -4.536      0.000      -0.018      -0.007
Employment_Employed, part-time                                                                -0.0227      0.004     -5.171      0.000      -0.031      -0.014
Employment_Independent contractor, freelancer, or self-employed                                0.0176      0.003      5.550      0.000       0.011       0.024
RemoteWork_Fully remote                                                                        0.0124      0.002      5.712      0.000       0.008       0.017
RemoteWork_Hybrid (some remote, some in-person)                                                0.0083      0.002      3.902      0.000       0.004       0.012
EdLevel_Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                           0.0087      0.003      2.539      0.011       0.002       0.015
EdLevel_Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                        0.0134      0.004      3.722      0.000       0.006       0.020
EdLevel_Other doctoral degree (Ph.D., Ed.D., etc.)                                             0.0272      0.005      5.380      0.000       0.017       0.037
EdLevel_Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)     0.0132      0.004      3.003      0.003       0.005       0.022
EdLevel_Some college/university study without earning a degree                                 0.0065      0.004      1.728      0.084      -0.001       0.014
OrgSize_10 to 19 employees                                                                    -0.0306      0.003    -10.536      0.000      -0.036      -0.025
OrgSize_10,000 or more employees                                                               0.0142      0.002      5.837      0.000       0.009       0.019
OrgSize_100 to 499 employees                                  

In [72]:
#creating dataframe with sorted values; from the least statistically significant to the most statistically significant
results_df = pd.DataFrame({'coef': results.params, 't': abs(results.tvalues), 'p-value': results.pvalues}) \
                        .sort_values(by=['t'])

# Interpretation of Results

OLS is a technique for estimating coefficients of **linear regression equations** which describe the **relationship** between one or more **independent quantitative variables** and a **dependent variable** *(simple or multiple linear regression)*. In this analysis, the dependent variable (y) is salary of people who do coding for living.

In the OLS method, those factors that have a **p-value < 0.05 are statistically significant**. The closer **t-value is to 0**, the more likely it **isn't a statistically significant**. Negative t-value are interpreted the same way as positive t-value, so we will take an **absolute value of t-value** to interprete t-value.

*The factors that have the most impact on salary:*
- **Continent_North America** - living in North America
- **Continent_Oceania** - living in Oceania
- **OrgSize_2 to 9 employees** - number of people that are employed by the company is from 2 to 9
- **YearsCodePro** - experience in professional coding
- **OpSys_macOS** - primary operating system is macOS

*The factors that have the least impact on salary:*
- **VersionControl_Other** - user doesn't work in Git nor SVN, but in some other version control system
- **DevType_Data scientist** - data scientist profession
- **NEWCTHaveWorked_Visual Studio** - working in Visual Studio
- **NEWCTHaveWorked_Sublime Text** - working in Sublime Text
- **NEWCTHaveWorked_Xcode** - working in Xcode

In [73]:
#results_df.tail(5) - The components that have the most impact on salary
#results_df.head(5) - The components that have the least impact on salary